In [ ]:
# для работы с папками
import os

# прогресс-бар
from tqdm.notebook import trange, tqdm

# регулярки
import re

# 
from datetime import timedelta, date

# для пауз при парсинге
from time import sleep

# для дафреймов
import pandas as pd

In [ ]:
file_path = 'C:/00_Projects/NYT/new/'

In [ ]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(file_path + 'decades/'):
    
    for filename in files:
        file_list += [filename]

In [ ]:
print(file_list)

In [ ]:
# считаю количество статей по годам 

article_number_year = pd.DataFrame()

for file_name in tqdm(file_list):
            
    work = pd.read_csv(file_path + 'decades/' + file_name) # загружаю датасет десятилетия
    
    temp_number = (work.groupby('year', as_index=False)
                   .agg({'clear': 'count'})
                   .rename(columns={'clear': 'article_number'}))
    
    temp_number['decade'] = file_name[4:13]
    
    article_number_year = article_number_year.append(temp_number)

In [ ]:
article_number_year.head()

In [ ]:
# записываю в файл

#article_number_year.to_excel(file_path + 'article_number.xlsx', encoding='utf-8', index=False)

In [ ]:
for file_name in tqdm(file_list):
            
    work = pd.read_csv(file_path + 'decades/' + file_name) # загружаю датасет десятилетия
    
    # список слов десятилетия
    word_list_all = [] # список для слов
    
    for words in work['new']:
        word_list_all += list(set(str(words).split())) # добавляю слова в список
        
    word_list_all = list(set(word_list_all)) # удаляю повторы
    word_list = [word for word in word_list_all if len(word) > 2] # удаляю слова из 1 и 2 символов
    
    # датафрейм слов десятилетия
    all_words = pd.DataFrame(word_list, columns=['word'])
        
    
    #
    # считаю частоты
    # датафрейм для частот десятилетия
    NYT_decade_words = pd.DataFrame() 
    
    year_list = list(work['year'].unique())
    
    for need_year in year_list:
       
        
        year_work = work[work['year'] == need_year]
        
        year_word_list_all = [] # список для слов года
        
        # добавляю слова в список
        for words in year_work['new']:
            year_word_list_all += list(set(str(words).split()))
            
        # датафрейм слов года
        year_words = pd.DataFrame(year_word_list_all, columns=['word'])
        
        # частота слова
        year_words['number'] = 1
        
        # суммирую частоты одинаковых слов
        year_NYT = (year_words.groupby('word', as_index=False)
                    .agg({'number': 'sum'}))
        
        # объединяю с словами десятилетия
        year_NYT_work = all_words.merge(year_NYT, how='left', on='word')
        
        # заполняю нули
        year_NYT_work['number'].fillna(0, inplace=True)
        
        # добавляю год
        year_NYT_work['year'] = need_year
        
        # добавляю в общий фрейм
        NYT_decade_words = NYT_decade_words.append(year_NYT_work)
        
    
    # добавляю декаду в датафрейм
    NYT_decade_words['decade'] = file_name[4:13]
    
    # записываю в файл
    NYT_decade_words.to_csv(file_path + 'words/words_' + file_name[4:13] + '.csv', encoding='utf-8', index=False)   